In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_csv('../data/real_estate_listings.csv')
df

,url,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
0,https://www.immowelt.de/expose/29fhx5b,Öl,Zentralheizung,249000.0,64.00,3.0,3.57,13505.0,1971.0,2.0,3890.625000
1,https://www.immowelt.de/expose/29hzk5u,Gas,"Fußbodenheizung, offener",1295000.0,136.06,4.0,3.57,10405.0,1900.0,5.0,9517.859768
2,https://www.immowelt.de/expose/292m95u,Gas,Etagenheizung,770000.0,120.00,4.0,3.57,12161.0,1900.0,5.0,6416.666667
3,https://www.immowelt.de/expose/29sh55u,Gas,Zentralheizung,349000.0,158.00,5.0,3.57,12103.0,1956.0,1.0,2208.860759
4,https://www.immowelt.de/expose/29r5y5t,Gas,Etagenheizung,494990.0,126.00,4.0,3.57,12157.0,1909.0,3.0,3928.492063
...,...,...,...,...,...,...,...,...,...,...,...
4937,https://www.immowelt.de/expose/2gd5v4m,Fernwärme,"Fußbodenheizung,",429000.0,78.00,3.0,3.57,13595.0,2000.0,1.0,5500.000000
4938,https://www.immowelt.de/expose/2ga5t4m,Gas,Zentralheizung,729000.0,150.00,4.0,3.57,13467.0,1990.0,1.0,4860.000000
4939,https://www.immowelt.de/expose/2f79g4a,Gas,Etagenheizung,695600.0,94.00,3.0,3.57,10115.0,1900.0,1.0,7400.000000
4940,https://www.immowelt.de/expose/2ej2m4d,Fernwärme,Minergie zertifiziert,839251.0,107.00,3.0,3.57,13591.0,1894.0,1.0,7843.467290


In [3]:
df.drop(['url'], axis = 1,inplace=True)

In [4]:
cols = ['zipcode', 'rooms', 'level','construction_year','price']
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4942 entries, 0 to 4941
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   energy             4942 non-null   object 
 1   heating            4942 non-null   object 
 2   price              4942 non-null   Int64  
 3   area               4942 non-null   float64
 4   rooms              4942 non-null   Int64  
 5   fee                4942 non-null   float64
 6   zipcode            4942 non-null   Int64  
 7   construction_year  4942 non-null   Int64  
 8   level              4942 non-null   Int64  
 9   price_per_area     4942 non-null   float64
dtypes: Int64(5), float64(3), object(2)
memory usage: 410.4+ KB


In [6]:
df.head()

,energy,heating,price,area,rooms,fee,zipcode,construction_year,level,price_per_area
0,Öl,Zentralheizung,249000,64.00,3,3.57,13505,1971,2,3890.625000
1,Gas,"Fußbodenheizung, offener",1295000,136.06,4,3.57,10405,1900,5,9517.859768
2,Gas,Etagenheizung,770000,120.00,4,3.57,12161,1900,5,6416.666667
3,Gas,Zentralheizung,349000,158.00,5,3.57,12103,1956,1,2208.860759
4,Gas,Etagenheizung,494990,126.00,4,3.57,12157,1909,3,3928.492063


In [7]:
print(df.isnull().sum())


energy               0
heating              0
price                0
area                 0
rooms                0
fee                  0
zipcode              0
construction_year    0
level                0
price_per_area       0
dtype: int64


In [8]:
df = df[(df['price'] > 0) & (df['area'] > 0)]


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4942 entries, 0 to 4941
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   energy             4942 non-null   object 
 1   heating            4942 non-null   object 
 2   price              4942 non-null   Int64  
 3   area               4942 non-null   float64
 4   rooms              4942 non-null   Int64  
 5   fee                4942 non-null   float64
 6   zipcode            4942 non-null   Int64  
 7   construction_year  4942 non-null   Int64  
 8   level              4942 non-null   Int64  
 9   price_per_area     4942 non-null   float64
dtypes: Int64(5), float64(3), object(2)
memory usage: 410.4+ KB


In [10]:
df = df.drop_duplicates()

In [11]:
neg_price = df[df['fee'] < 0]
print(neg_price)


     energy heating   price   area  rooms  fee  zipcode  construction_year  \
2181     na      na  230000  66.78      2 -3.5    12043               1900   

      level  price_per_area  
2181      1     3444.144954  


In [12]:
df = df[df['fee'] >= 0]


In [13]:
print(df['heating'].unique())

['Zentralheizung' 'Fußbodenheizung, offener ' 'Etagenheizung' 'na'
 'Fußbodenheizung, ' 'Kamin, ' 'Fußbodenheizung' 'Heizungsart' 'Wasser-'
 'Kf' 'Luft-/' 'Kamin' 'Wärmepumpe' 'Niedrigenergiehaus' 'Etagenheizung, '
 'Ofen, ' 'Etagenheizung, offener ' 'Wärmepumpe, ' 'Fertighaus'
 'Wärmepumpe, offener ' 'Kohle, ' 'Heizungsartoffener '
 'Minergie zertifiziert']


In [14]:
df['heating_raw'] = df['heating']

In [15]:
df['heating_clean'] = (
    df['heating_raw']
    .str.lower()
    .str.strip()
    .str.replace(r',$', '', regex=True)
    .str.strip()
)

In [16]:
heating_map = {
    'zentralheizung': 'zentralheizung',
    'fußbodenheizung': 'fußbodenheizung',
    'fußbodenheizung offener': 'fußbodenheizung',
    'fußbodenheizung, offener':'fußbodenheizung',
    'fußbodenheizung':'fußbodenheizung',
    'etagenheizung': 'etagenheizung',
    'etagenheizung offener': 'etagenheizung',
    'kamin': 'kamin',
    'kohle': 'kohle',
    'wärmepumpe': 'wärmepumpe',
    'wärmepumpe, offener' : 'wärmepumpe',
    'minergie zertifiziert': 'minergie',
    'etagenheizung, offener' : 'etagenheizung',

    'kamin':'other',
    'wärmepumpe':'other',
    'kf':'other',
    'minergie':'other',
    'ofen':'other',
    'kohle':'other',
    'fertighaus': 'other',
    'niedrigenergiehaus': 'other',
    'wasser-': 'other',
    'k f': 'kf',
    'luft-/': 'other',
    'heizungsart': 'other',
    'heizungsartoffener': 'other',
    'na': 'missing'
}
df['heating_clean'] = df['heating_clean'].replace(heating_map)


In [17]:
print(df['heating_clean'].unique())

['zentralheizung' 'fußbodenheizung' 'etagenheizung' 'missing' 'other'
 'wärmepumpe' 'minergie']


In [18]:
print(df['heating_clean'].nunique())

7


In [44]:
print(df.columns)

Index(['energy', 'heating', 'price', 'area', 'rooms', 'fee', 'zipcode',
       'construction_year', 'level', 'price_per_area', 'heating_raw',
       'heating_clean'],
      dtype='object')


In [19]:
print(df['heating_clean'].value_counts())

heating_clean
zentralheizung     1733
missing            1723
etagenheizung       360
fußbodenheizung     312
other               193
minergie              2
wärmepumpe            1
Name: count, dtype: int64


In [56]:
df['energy_clean'].unique()

array(['öl', 'gas', 'na', 'fernwärme', 'elektro', 'fußbodenheizung',
       'flüssiggas', 'luft-/', 'blockheizkraftwerk', 'holz', 'erdwärme',
       'solar', 'niedrigenergiehaus', 'pellets',
       'fußbodenheizung, offener'], dtype=object)

['Öl' 'Gas' 'na' 'Fernwärme' 'Fernwärme, ' 'Elektro' 'Elektro, ' 'Gas, '
 'Fußbodenheizung, ' 'Flüssiggas' 'Luft-/' 'Blockheizkraftwerk, ' 'Holz, '
 'Erdwärme' 'Solar' 'Niedrigenergiehaus' 'Pellets' 'Blockheizkraftwerk'
 'Erdwärme, ' 'Fußbodenheizung, offener ']


In [52]:
df['energy_clean'] = (
    df['energy']
    .str.lower()
    .str.strip()
    .str.replace(r',$', '', regex=True)
    .str.strip()
)

In [110]:
energy_map = {
    'öl': 'öl',
    'gas': 'gas',
    'elektro': 'elektro',
    'flüssiggas': 'flüssiggas',
    'blockheizkraftwerk': 'blockheizkraftwerk',
    'holz': 'holz',
    'erdwärme': 'erdwärme',
    'solar': 'solar',
    'pellets': 'pellets',
    'fernwärme': 'fernwärme',

    'luft-/': 'other',  # unclear category, group as other
    'niedrigenergiehaus': 'other',  # building type, not energy source
    'fußbodenheizung': 'other',  # heating type, not energy source
    'fußbodenheizung, offener': 'other', 
    'na': 'missing',
    '': 'missing',
    None: 'missing'
}


df['energy_clean'] = df['energy_clean'].str.lower().str.strip().str.rstrip(',')
df['energy_clean'] = df['energy_clean'].replace(heating_map)

In [112]:
print(df['energy_clean'].unique())

['öl' 'gas' 'missing' 'fernwärme' 'elektro' 'fußbodenheizung' 'flüssiggas'
 'other' 'blockheizkraftwerk' 'holz' 'erdwärme' 'solar' 'pellets']


In [92]:
print(df['energy_clean'].nunique())

13


In [104]:
print(df['energy_clean'].value_counts())

energy_clean
missing               1723
gas                   1345
fernwärme              747
öl                     390
elektro                 38
fußbodenheizung         25
other                   19
blockheizkraftwerk      13
solar                   12
erdwärme                 6
pellets                  3
flüssiggas               2
holz                     1
Name: count, dtype: int64


In [114]:
print(df.loc[df['energy_clean'].str.contains('fußbodenheizung', na=False), 'energy_clean'].unique())


['fußbodenheizung']


In [116]:
df.loc[df['energy_clean'].str.contains('fußbodenheizung', na=False), 'energy_clean'] = 'other'


In [118]:
print(df['energy_clean'].unique())

['öl' 'gas' 'missing' 'fernwärme' 'elektro' 'other' 'flüssiggas'
 'blockheizkraftwerk' 'holz' 'erdwärme' 'solar' 'pellets']


In [120]:
print(df['energy_clean'].nunique())

12


In [122]:
print(df['energy_clean'].value_counts())

energy_clean
missing               1723
gas                   1345
fernwärme              747
öl                     390
other                   44
elektro                 38
blockheizkraftwerk      13
solar                   12
erdwärme                 6
pellets                  3
flüssiggas               2
holz                     1
Name: count, dtype: int64


In [124]:
threshold = 50
counts = df['energy_clean'].value_counts()
rare_categories = counts[counts < threshold].index
df['energy_clean'] = df['energy_clean'].replace(rare_categories, 'other')

In [126]:
print(df['energy_clean'].value_counts())

energy_clean
missing      1723
gas          1345
fernwärme     747
öl            390
other         119
Name: count, dtype: int64
